In [1]:
from pyspark.ml import feature
from pyspark.ml import clustering
from pyspark.ml import Pipeline
from pyspark.sql import functions as fn
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.sql import functions as sf
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.feature import RegexTokenizer
import requests
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import concat, col, lit, lower
from pyspark.sql.functions import isnan, when, count, col, isnull
from pyspark.sql.functions import concat_ws
from  pyspark.sql.functions import abs
# seting master("local[*]") enables multicore processing on all available logical cores on your machine
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [2]:
# Do not delete or change this cell

import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

In [3]:
airlines_df = spark.read.csv(get_training_filename('airlines.csv'), header=True, inferSchema=True)
airports_df = spark.read.csv(get_training_filename('airports.csv'), header=True, inferSchema=True)
flights_df = spark.read.csv(get_training_filename('flights.csv'), header=True, inferSchema=True)

# shape = ((raw_ds_programs_text_df.count(), len(raw_ds_programs_text_df.columns)))
# print('The shape of raw_ds_programs_text_df:', shape)

# raw_ds_programs_text_df.toPandas().head()


In [4]:
shape = ((flights_df.count(), len(flights_df.columns)))
print('The shape of flights_df:', shape)

The shape of flights_df: (5819079, 31)

In [5]:
flights_df.select([count(when(isnull(c), c)).alias(c) for c in flights_df.columns]).show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
 0| 0| 0| 0| 0| 0| 14721| 0| 0| 0| 86153| 86153| 89047| 89047| 6| 105071| 105071| 0| 92513| 92513| 0| 92513| 105071| 0| 0| 5729195| 4755640| 4755640| 4755640| 4755640| 4755640|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+

In [6]:
flights_df.printSchema()

root
-- YEAR: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- AIRLINE: string (nullable = true)
-- FLIGHT_NUMBER: integer (nullable = true)
-- TAIL_NUMBER: string (nullable = true)
-- ORIGIN_AIRPORT: string (nullable = true)
-- DESTINATION_AIRPORT: string (nullable = true)
-- SCHEDULED_DEPARTURE: integer (nullable = true)
-- DEPARTURE_TIME: integer (nullable = true)
-- DEPARTURE_DELAY: integer (nullable = true)
-- TAXI_OUT: integer (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- SCHEDULED_TIME: integer (nullable = true)
-- ELAPSED_TIME: integer (nullable = true)
-- AIR_TIME: integer (nullable = true)
-- DISTANCE: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: integer (nullable = true)
-- SCHEDULED_ARRIVAL: integer (nullable = true)
-- ARRIVAL_TIME: integer (nullable = true)
-- ARRIVAL_DELAY: integer (nullable = true)
-- DIVERTED: integer (nullable = true)
-- CANCELLED: integer (nullable = true)
-- CANCELLATION_REASON: string (nullable = true)
-- AIR_SYSTEM_DELAY: integer (nullable = true)
-- SECURITY_DELAY: integer (nullable = true)
-- AIRLINE_DELAY: integer (nullable = true)
-- LATE_AIRCRAFT_DELAY: integer (nullable = true)
-- WEATHER_DELAY: integer (nullable = true)

In [7]:
flights_df = flights_df.select('MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'ARRIVAL_DELAY', 'CANCELLED')

In [8]:
flights_df.printSchema()

root
-- MONTH: integer (nullable = true)
-- DAY: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- AIRLINE: string (nullable = true)
-- ORIGIN_AIRPORT: string (nullable = true)
-- DESTINATION_AIRPORT: string (nullable = true)
-- SCHEDULED_DEPARTURE: integer (nullable = true)
-- DEPARTURE_DELAY: integer (nullable = true)
-- DISTANCE: integer (nullable = true)
-- SCHEDULED_ARRIVAL: integer (nullable = true)
-- ARRIVAL_DELAY: integer (nullable = true)
-- CANCELLED: integer (nullable = true)

In [9]:
flights_df.select([count(when(isnull(c), c)).alias(c) for c in flights_df.columns]).show()

+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+-------------+---------+
MONTH|DAY|DAY_OF_WEEK|AIRLINE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_DELAY|DISTANCE|SCHEDULED_ARRIVAL|ARRIVAL_DELAY|CANCELLED|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+-------------+---------+
 0| 0| 0| 0| 0| 0| 0| 86153| 0| 0| 105071| 0|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+-------------+---------+

In [10]:
from pyspark.sql import functions as fn
flights_df.groupBy('CANCELLED').agg(fn.count('*')).show()

+---------+--------+
CANCELLED|count(1)|
+---------+--------+
 1| 89884|
 0| 5729195|
+---------+--------+

In [11]:
shape = ((flights_df.count(), len(flights_df.columns)))
print('The shape of flights_df:', shape)

The shape of flights_df: (5729195, 12)

In [12]:
flights_df = flights_df.filter((fn.col('CANCELLED')==0))

In [13]:
flights_df = flights_df.withColumn("Flight_Delayed", fn.when(fn.col("DEPARTURE_DELAY")<10, 0).otherwise(1))

In [14]:
flights_df.select("DISTANCE").rdd.max()[0]

Out[26]: 4983

In [15]:
flights_df.select("DISTANCE").rdd.min()[0]

Out[27]: 31

In [16]:
from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=[ 0, 100, 1000, float('Inf') ],inputCol="DISTANCE", outputCol="Distance_Bucket")
flights_df = bucketizer.setHandleInvalid("keep").transform(flights_df)

In [17]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

t = {0.0:"Short", 1.0: "Medium", 2.0:"Long"}
udf_foo = udf(lambda x: t[x], StringType())
flights_df = flights_df.withColumn("Flight_Distance", udf_foo("Distance_Bucket"))

In [18]:
flights_df.limit(50).toPandas()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,CANCELLED,Flight_Delayed,Distance_Bucket,Flight_Distance
0,1,1,4,AS,ANC,SEA,5,-11,1448,430,-22,0,0,2.0,Long
1,1,1,4,AA,LAX,PBI,10,-8,2330,750,-9,0,0,2.0,Long
2,1,1,4,US,SFO,CLT,20,-2,2296,806,5,0,0,2.0,Long
3,1,1,4,AA,LAX,MIA,20,-5,2342,805,-9,0,0,2.0,Long
4,1,1,4,AS,SEA,ANC,25,-1,1448,320,-21,0,0,2.0,Long
5,1,1,4,DL,SFO,MSP,25,-5,1589,602,8,0,0,2.0,Long
6,1,1,4,NK,LAS,MSP,25,-6,1299,526,-17,0,0,2.0,Long
7,1,1,4,US,LAX,CLT,30,14,2125,803,-10,0,1,2.0,Long
8,1,1,4,AA,SFO,DFW,30,-11,1464,545,-13,0,0,2.0,Long
9,1,1,4,DL,LAS,ATL,30,3,1747,711,-15,0,0,2.0,Long


In [19]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="AIRLINE", outputCol="Airline_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

In [20]:
from pyspark.sql import functions as fn
flights_df.groupBy('Airline_Numeric').agg(fn.count('*')).show()

+---------------+--------+
Airline_Numeric|count(1)|
+---------------+--------+
 8.0| 194648|
 0.0| 1245812|
 7.0| 262772|
 1.0| 872057|
 4.0| 556746|
 11.0| 90248|
 3.0| 578393|
 2.0| 715065|
 10.0| 115375|
 13.0| 61369|
 6.0| 279607|
 5.0| 509150|
 9.0| 171852|
 12.0| 76101|
+---------------+--------+

In [21]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCol="Airline_Numeric", outputCol="Airline_OHE")
flights_df= encoder.transform(flights_df)

In [22]:
flights_df.limit(10).toPandas()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,CANCELLED,Flight_Delayed,Distance_Bucket,Flight_Distance,Airline_Numeric,Airline_OHE
0,1,1,4,AS,ANC,SEA,5,-11,1448,430,-22,0,0,2.0,Long,9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,1,4,AA,LAX,PBI,10,-8,2330,750,-9,0,0,2.0,Long,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,1,4,US,SFO,CLT,20,-2,2296,806,5,0,0,2.0,Long,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
3,1,1,4,AA,LAX,MIA,20,-5,2342,805,-9,0,0,2.0,Long,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,1,4,AS,SEA,ANC,25,-1,1448,320,-21,0,0,2.0,Long,9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,1,1,4,DL,SFO,MSP,25,-5,1589,602,8,0,0,2.0,Long,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,1,1,4,NK,LAS,MSP,25,-6,1299,526,-17,0,0,2.0,Long,10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,1,1,4,US,LAX,CLT,30,14,2125,803,-10,0,1,2.0,Long,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
8,1,1,4,AA,SFO,DFW,30,-11,1464,545,-13,0,0,2.0,Long,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,1,1,4,DL,LAS,ATL,30,3,1747,711,-15,0,0,2.0,Long,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [23]:
flights_df.select('Airline_OHE').take(5)


Out[75]: [Row(Airline_OHE=SparseVector(13, {9: 1.0})),
 Row(Airline_OHE=SparseVector(13, {2: 1.0})),
 Row(Airline_OHE=SparseVector(13, {8: 1.0})),
 Row(Airline_OHE=SparseVector(13, {2: 1.0})),
 Row(Airline_OHE=SparseVector(13, {9: 1.0}))]

In [24]:
indexer = StringIndexer(inputCol="ORIGIN_AIRPORT", outputCol="OA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

In [25]:
encoder = OneHotEncoder(inputCol="OA_Numeric", outputCol="Origin_Airport_OHE")
flights_df= encoder.transform(flights_df)

In [26]:
flights_df.limit(10).toPandas()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,CANCELLED,Flight_Delayed,Distance_Bucket,Flight_Distance,Airline_Numeric,Airline_OHE,OA_Numeric,Origin_Airport_OHE
0,1,1,4,AS,ANC,SEA,5,-11,1448,430,-22,0,0,2.0,Long,9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",65.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,1,4,AA,LAX,PBI,10,-8,2330,750,-9,0,0,2.0,Long,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,1,4,US,SFO,CLT,20,-2,2296,806,5,0,0,2.0,Long,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",6.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
3,1,1,4,AA,LAX,MIA,20,-5,2342,805,-9,0,0,2.0,Long,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,1,4,AS,SEA,ANC,25,-1,1448,320,-21,0,0,2.0,Long,9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,1,1,4,DL,SFO,MSP,25,-5,1589,602,8,0,0,2.0,Long,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
6,1,1,4,NK,LAS,MSP,25,-6,1299,526,-17,0,0,2.0,Long,10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
7,1,1,4,US,LAX,CLT,30,14,2125,803,-10,0,1,2.0,Long,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
8,1,1,4,AA,SFO,DFW,30,-11,1464,545,-13,0,0,2.0,Long,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
9,1,1,4,DL,LAS,ATL,30,3,1747,711,-15,0,0,2.0,Long,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."


In [27]:
indexer = StringIndexer(inputCol="DESTINATION_AIRPORT", outputCol="DA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

In [28]:
encoder = OneHotEncoder(inputCol="DA_Numeric", outputCol="Destination_Airport_OHE")
flights_df= encoder.transform(flights_df)

In [29]:
flights_df.limit(10).toPandas()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,CANCELLED,Flight_Delayed,Distance_Bucket,Flight_Distance,Airline_Numeric,Airline_OHE,OA_Numeric,Origin_Airport_OHE,DA_Numeric,Destination_Airport_OHE
0,1,1,4,AS,ANC,SEA,5,-11,1448,430,-22,0,0,2.0,Long,9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",65.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,1,4,AA,LAX,PBI,10,-8,2330,750,-9,0,0,2.0,Long,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",52.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,1,4,US,SFO,CLT,20,-2,2296,806,5,0,0,2.0,Long,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",6.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",14.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1,1,4,AA,LAX,MIA,20,-5,2342,805,-9,0,0,2.0,Long,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",24.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,1,4,AS,SEA,ANC,25,-1,1448,320,-21,0,0,2.0,Long,9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",66.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,1,1,4,DL,SFO,MSP,25,-5,1589,602,8,0,0,2.0,Long,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,1,1,4,NK,LAS,MSP,25,-6,1299,526,-17,0,0,2.0,Long,10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,1,1,4,US,LAX,CLT,30,14,2125,803,-10,0,1,2.0,Long,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",14.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,1,1,4,AA,SFO,DFW,30,-11,1464,545,-13,0,0,2.0,Long,2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",2.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,1,1,4,DL,LAS,ATL,30,3,1747,711,-15,0,0,2.0,Long,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [30]:
shape = ((flights_df.count(), len(flights_df.columns)))
print('The shape of flights_df:', shape)

The shape of flights_df: (5729195, 21)

In [31]:
flights_df = flights_df.drop('ARRIVAL_DELAY')

In [32]:
flights_df.printSchema()

root
-- MONTH: integer (nullable = true)
-- DAY: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- AIRLINE: string (nullable = true)
-- ORIGIN_AIRPORT: string (nullable = true)
-- DESTINATION_AIRPORT: string (nullable = true)
-- SCHEDULED_DEPARTURE: integer (nullable = true)
-- DEPARTURE_DELAY: integer (nullable = true)
-- DISTANCE: integer (nullable = true)
-- SCHEDULED_ARRIVAL: integer (nullable = true)
-- CANCELLED: integer (nullable = true)
-- Flight_Delayed: integer (nullable = false)
-- Distance_Bucket: double (nullable = true)
-- Flight_Distance: string (nullable = true)
-- Airline_Numeric: double (nullable = false)
-- Airline_OHE: vector (nullable = true)
-- OA_Numeric: double (nullable = false)
-- Origin_Airport_OHE: vector (nullable = true)
-- DA_Numeric: double (nullable = false)
-- Destination_Airport_OHE: vector (nullable = true)

In [33]:
training_df, testing_df = flights_df.randomSplit([0.8, 0.2])

In [34]:
testing_df.select([count(when(isnull(c), c)).alias(c) for c in training_df.columns]).show()

+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+-----------+----------+------------------+----------+-----------------------+
MONTH|DAY|DAY_OF_WEEK|AIRLINE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_DELAY|DISTANCE|SCHEDULED_ARRIVAL|CANCELLED|Flight_Delayed|Distance_Bucket|Flight_Distance|Airline_Numeric|Airline_OHE|OA_Numeric|Origin_Airport_OHE|DA_Numeric|Destination_Airport_OHE|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+-----------+----------+------------------+----------+-----------------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+-----------+----------+------------------+----------+-----------------------+

In [35]:

training_df.groupBy('Flight_Delayed').agg(fn.count('*')).show()

+--------------+--------+
Flight_Delayed|count(1)|
+--------------+--------+
 1| 1034445|
 0| 3549295|
+--------------+--------+

In [36]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(
    inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "CANCELLED"], outputCol="features")

In [37]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline

In [38]:
lr = LogisticRegression(featuresCol='features', labelCol='Flight_Delayed', regParam=0.1)

In [39]:
lr_pipeline = Pipeline(stages=[va, lr]).fit(training_df)

In [40]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
bce = BinaryClassificationEvaluator(labelCol='Flight_Delayed', metricName='areaUnderROC')

In [41]:
bce.evaluate(lr_pipeline.transform(testing_df))

Out[126]: 0.6601326918638712

In [42]:
rf = RandomForestClassifier(featuresCol='features', labelCol='Flight_Delayed')

In [43]:
rf_pipeline = Pipeline(stages=[va, rf]).fit(training_df)

In [44]:
bce.evaluate(rf_pipeline.transform(testing_df))

Out[29]: 0.5254238211864535

In [45]:
rf_model = rf_pipeline.stages[-1]

In [46]:
pd.DataFrame(list(zip(flights_df.columns[2:], rf_model.featureImportances.toArray())),
            columns = ['column', 'weight']).sort_values('weight', ascending=False)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3835369770817059> in <module> 
 ----> 1 pd.DataFrame(list(zip(credit_score_df.columns[2:], rf_model.featureImportances.toArray())),
 2 columns = ['column', 'weight']).sort_values('weight', ascending=False)

 NameError : name 'credit_score_df' is not defined

In [47]:
rf_model.featureImportances.toArray()

Out[120]: array([0.13991277, 0.00017674, 0.00069704, ..., 0. , 0.01935258,
 0. ])

In [48]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(featuresCol='features', labelCol='Flight_Delayed')

In [49]:
gbt_pipeline = Pipeline(stages=[va, gbt]).fit(training_df)

In [50]:
bce.evaluate(gbt_pipeline.transform(testing_df))

java.io.IOException: Connection failed
	at com.databricks.rpc.Jetty9Client$$anon$1.handleError(Jetty9Client.scala:468)
	at com.databricks.rpc.Jetty9Client$$anon$1.onFailure(Jetty9Client.scala:395)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:177)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:169)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.notifyFailureComplete(HttpExchange.java:268)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.abort(HttpExchange.java:240)
	at shaded.v9_4.org.eclipse.jetty.client.HttpConversation.abort(HttpConversation.java:149)
	at shaded.v9_4.org.eclipse.jetty.client.HttpRequest.abort(HttpRequest.java:768)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.abort(HttpDestination.java:453)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.failed(HttpDestination.java:234)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectionPool$1.failed(AbstractConnectionPool.java:140)
	at shaded.v9_4.org.eclipse.jetty.util.Promise$Wrapper.failed(Promise.java:136)
	at shaded.v9_4.org.eclipse.jetty.client.HttpClient$1$1.failed(HttpClient.java:612)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport.connectFailed(AbstractConnectorHttpClientTransport.java:138)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport$ClientSelectorManager.connectionFailed(AbstractConnectorHttpClientTransport.java:188)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$Connect.failed(ManagedSelector.java:822)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:254)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1400(ManagedSelector.java:62)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:543)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:401)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:360)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:184)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:171)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:129)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:367)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:782)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:914)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:714)
	at shaded.v9_4.org.eclipse.jetty.io.SelectorManager.doFinishConnect(SelectorManager.java:355)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:232)
	... 11 more